In [4]:
# prompt: upload data folder from drive

from google.colab import drive
drive.mount('/content/drive')

# After mounting, you can access files in your Google Drive
# For example, to list contents of a folder named 'data' in your Drive:
!ls /content/drive/MyDrive/data

Mounted at /content/drive
Flamingo  King_penguin	Macaw  ostrich	Owl  Peacock


In [5]:
import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
train_dir = '/content/drive/MyDrive/data'

In [7]:
# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=20,
    zoom_range=0.2, horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=(224, 224), class_mode='categorical')

Found 600 images belonging to 6 classes.


In [12]:
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable  = False


model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(6, activation='softmax')


    ])

In [13]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           774 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,726 (9.24 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [14]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(train_generator, epochs=5)

Epoch 1/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9425 - loss: 0.1521
Epoch 2/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9701 - loss: 0.1103
Epoch 3/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9758 - loss: 0.0920
Epoch 4/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step - accuracy: 0.9688 - loss: 0.1031
Epoch 5/5
19/19 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.9764 - loss: 0.0902


In [18]:
model.save('model.h5')

In [19]:
import gradio as gr
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [20]:
model = load_model('model.h5')
class_names = list(train_generator.class_indices.keys())

In [21]:
class_names

['Flamingo', 'King_penguin', 'Macaw', 'Owl', 'Peacock', 'ostrich']

In [22]:


def predict_img(img):
    # Resize image to 224x224
    img = img.resize((224, 224))

    # Convert PIL Image to NumPy array and normalize
    img_arr = np.array(img) / 255.0

    # Expand dimensions to match model input: (1, 224, 224, 3)
    img_arr = np.expand_dims(img_arr, axis=0)

    # Predict using the model
    prediction = model.predict(img_arr)

    # Return the predicted class name
    return class_names[np.argmax(prediction[0])]


In [24]:
gr.Interface(
    fn=predict_img,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=1)
).launch(share=True, debug=True)


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9265e1d8ca7d4e86fc.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9265e1d8ca7d4e86fc.gradio.live
